In [ ]:
### Goal: 在104輸入職業關鍵字，輸出相應的工具技能需求
# 1. look pattern
# 在104搜尋"軟體師工程師"
# https://www.104.com.tw/jobbank/joblist/joblist.cfm?keyword=%E8%BB%9F%E9%AB%94%E5%B7%A5%E7%A8%8B%E5%B8%AB&jobsource=n104bank1&ro=0&order=1
# https://www.104.com.tw/jobbank/joblist/joblist.cfm?keyword=軟體工程師&jobsource=n104bank1&ro=0&order=1
# 中間那一串keyword打中文也可以自動轉換encoding，所以上兩個一樣
# 點選第一項job    
# https://www.104.com.tw/job/?jobno=5jxqt&jobsource=104_hotorder
# 點選第二項
# https://www.104.com.tw/job/?jobno=4qh2a&jobsource=104_hotorder
# https://www.104.com.tw/job/?jobno=59t39&jobsource=104_hotorder
# -> 好像只有中間的編碼不同，先爬取中間的編碼

In [ ]:
# 最後輸出結果

# ubuntu@ubuntu-VirtualBox:~/Workspace/pythonFiles$ python search104.py
# 請輸入工作關鍵字，並為您輸出排名前十工具：軟體工程師
# [('C++', 629), ('C', 477), ('Java', 466), ('C#', 461), ('JavaScript', 355), ('Linux', 301), ('MS SQL', 294), ('HTML', 242), ('ASP.NET', 219), ('jQuery', 195)]
# Cost seconds: 1300.1823642253876

# 請輸入工作關鍵字，並為您輸出排名前十工具：大數據
# [('Excel', 480), ('Word', 463), ('PowerPoint', 381), ('Outlook', 271), ('Java', 153), ('JavaScript', 126), ('HTML', 104), ('MS SQL', 101), ('Adobe Photoshop', 95), ('Linux', 92)]
# Cost seconds: 1065.3629341125488


In [ ]:
# 2. tools import
!pip install requests
!pip install BeautifulSoup4
!pip install lxml

In [52]:
import requests
import time
from bs4 import BeautifulSoup
from collections import Counter

In [10]:
# 3. 拆解動作 requests.get
res = requests.get("https://www.104.com.tw/jobbank/joblist/joblist.cfm?keyword=%E8%BB%9F%E9%AB%94%E5%B7%A5%E7%A8%8B%E5%B8%AB&jobsource=n104bank1&ro=0&order=1")

In [10]:
# 查詢回應情況
res.status_code

200

In [11]:
# Use BeautifulSoup as my text parser
soup = BeautifulSoup(res.text, 'lxml')

In [12]:
# Inspect the pattern inside each job block, find the tags of the job href
# .job_name > a href 
print(len(soup.select(".job_name > a")))

30


In [13]:
# Use ['href'] take out the specific part from the reference
for link in soup.select(".job_name > a"):
    print(link['href'][12:17])

5jxqt
2ti1i
2sa50
5hqoi
4qh2a
50qor
5gc0n
5h2my
4zwgk
4r2uj
4x4sc
475dt
3mi7n
2uggc
4lgy0
2gr2o
4wj8d
3o65&
58it4
36620
40ojf
3u816
43pgg
49pil
3z9lq
azkm&
52bye
31ppu
59jh0
5fdfv


In [22]:
job_url = "https://www.104.com.tw/job/?jobno=5jxqt&jobsource=104_hotorder"
job_res = requests.get(job_url)
job_soup = BeautifulSoup(job_res.text, 'lxml')
print(job_soup.select(".tool")[0])

<dd class="tool"><a href="https://learn.104.com.tw/cfdocs/edu/search/tool_class.cfm?from=104tool&amp;no=12001001007" target="_blank" title="想學Linux嗎">Linux</a>、<a href="https://learn.104.com.tw/cfdocs/edu/search/tool_class.cfm?from=104tool&amp;no=12001001042" target="_blank" title="想學Android嗎">Android</a>、<a href="https://learn.104.com.tw/cfdocs/edu/search/tool_class.cfm?from=104tool&amp;no=12001001043" target="_blank" title="想學iOS嗎">iOS</a>、<a href="https://learn.104.com.tw/cfdocs/edu/search/tool_class.cfm?from=104tool&amp;no=12001003008" target="_blank" title="想學C嗎">C</a>、<a href="https://learn.104.com.tw/cfdocs/edu/search/tool_class.cfm?from=104tool&amp;no=12001003010" target="_blank" title="想學C++嗎">C++</a>、<a href="https://learn.104.com.tw/cfdocs/edu/search/tool_class.cfm?from=104tool&amp;no=12001003025" target="_blank" title="想學Java嗎">Java</a>、<a href="https://learn.104.com.tw/cfdocs/edu/search/tool_class.cfm?from=104tool&amp;no=12001003075" target="_blank" title="想學Objective-C嗎">

In [24]:
# select the tool name from each job page
toolName = {}
for link in soup.select(".job_name > a"):
    tn_code = link['href'][12:17]  #tn code -> tool name code
    job_url = "https://www.104.com.tw/job/?jobno="+tn_code+"&jobsource=104_hotorder"
    job_res = requests.get(job_url)
    job_soup = BeautifulSoup(job_res.text, 'lxml')

    for tools in job_soup.select(".tool")[0].select("a"):
        tn = tools.text #tn -> tool name
        if tn in toolName:
            toolName[tn] += 1
        else:
            toolName[tn] = 1
toolName
# this result only for the front 30 pieces

{'ADO': 1,
 'AJAX': 2,
 'ASP.NET': 3,
 'Access': 1,
 'Android': 3,
 'ArcGis': 1,
 'C': 8,
 'C#': 8,
 'C++': 7,
 'CVS': 1,
 'DB2': 1,
 'DHTML': 1,
 'Delphi': 1,
 'Excel': 1,
 'FreeBSD': 1,
 'HTML': 2,
 'J2EE': 2,
 'J2SE': 1,
 'JDBC': 1,
 'JSP': 4,
 'Java': 9,
 'JavaScript': 5,
 'Linux': 5,
 'MFC': 1,
 'MS SQL': 4,
 'MapGIS': 1,
 'MySQL': 5,
 'ODBC': 1,
 'OOP': 1,
 'Objective-C': 2,
 'Oracle': 3,
 'Oracle ERP': 1,
 'PHP': 3,
 'PL/SQL': 1,
 'PostgreSQL': 1,
 'Project': 1,
 'Python': 2,
 'SWIFT': 1,
 'Spring': 1,
 'Struts': 1,
 'Systems Analysis': 1,
 'Visio': 1,
 'Visual Basic .net': 1,
 'Visual C#': 3,
 'Visual C++': 1,
 'Visual Studio': 2,
 'Visual Studio .net': 3,
 'Win32': 1,
 'WinForm': 1,
 'Windows 2000': 1,
 'Windows 7': 1,
 'Windows 8': 1,
 'Windows Server 2008': 1,
 'Windows Server 2012': 1,
 'Windows XP': 1,
 'Word': 1,
 'XML': 1,
 'iOS': 2,
 'jQuery': 4,
 '中文打字20~50': 1,
 '鼎新': 1}

In [19]:
# find the total pages:
pagesNum = len(soup.select(".box_page_top > ul > li"))
pagesNum

150

In [ ]:
# 點選page 1
https://www.104.com.tw/jobbank/joblist/joblist.cfm?jobsource=n104bank1&ro=0&keyword=省略&order=1&asc=0&page=1
# p2
https://www.104.com.tw/jobbank/joblist/joblist.cfm?jobsource=n104bank1&ro=0&keyword=省略&order=1&asc=0&page=2&psl=N_A

In [11]:
# all the components are prepared! Let's write it as a query script

import requests
import time
from bs4 import BeautifulSoup
from collections import Counter

start_time = time.time()
def jobKeyword2tools():
    keyword = input("請輸入工作關鍵字，並為您輸出排名前十工具：")
    toolName = Counter()
    res = requests.get("https://www.104.com.tw/jobbank/joblist/joblist.cfm?keyword="+keyword+"&jobsource=n104bank1&ro=0&order=1")
    soup = BeautifulSoup(res.text, 'lxml')
#     jobAmount1page = len(soup.select(".job_name"))
    pagesAmount = len(soup.select(".box_page_top > ul > li"))
    for page in range(pagesAmount):
#     for page in range(40, 42):
        print(page+1)
        res = requests.get("https://www.104.com.tw/jobbank/joblist/joblist.cfm?jobsource=n104bank1&ro=0&keyword="+keyword+"&order=1&asc=0&page="+(str)(page+1)+"&psl=N_A")
        soup = BeautifulSoup(res.text, 'lxml')
        count=0
        for link in soup.select(".job_name > a"): #inside of each job page
            count += 1
            if link['href'][6:11] != "jobno":
                print(link['href'])
                pass
            else:   
                tn_code = link['href'][12:17]  #tn code -> tool name code
#             print(tn_code)
                print("a link"+(str)(count))
                job_url = "https://www.104.com.tw/job/?jobno="+tn_code+"&jobsource=104_hotorder"
                job_res = requests.get(job_url)
                job_soup = BeautifulSoup(job_res.text, 'lxml')
                for tools in job_soup.select(".tool")[0].select("a"):
                    tn = tools.text #tn -> tool name
                    if tn in toolName:
                        toolName[tn] += 1
                    else:
                        toolName[tn] = 1    
    return toolName.most_common(10)
    
print(jobKeyword2tools())   
print("Cost seconds: %s"%(time.time()-start_time))

請輸入工作關鍵字，並為您輸出排名前十工具：大數據
1
a link1
a link2
a link3
a link4
a link5
a link6
a link7
a link8
a link9
a link10
a link11
a link12
a link13
a link14
a link15
a link16
a link17
a link18
a link19
a link20
a link21
a link22
2
a link1
a link2
a link3
a link4
a link5
a link6
a link7
a link8
a link9
a link10
a link11
a link12
a link13
a link14
a link15
a link16
a link17
a link18
a link19
a link20
3
a link1
a link2
a link3
a link4
a link5
a link6
a link7
a link8
a link9
a link10
a link11
a link12
a link13
a link14
a link15
a link16
a link17
a link18
a link19
a link20
4
a link1
a link2
a link3
a link4
a link5
a link6
a link7
a link8
a link9
a link10
a link11
a link12
a link13
a link14
a link15
a link16
a link17
a link18
a link19
a link20
5
a link1
a link2
a link3
a link4
a link5
a link6
a link7
a link8
a link9
a link10
a link11
a link12
a link13
a link14
a link15
a link16
a link17
a link18
a link19
a link20
6
a link1
a link2
a link3
a link4
a link5
a link6
a link7
a link8
a link9
a link10
a link11


a link18
a link19
a link20
47
a link1
a link2
a link3
a link4
a link5
a link6
a link7
a link8
a link9
a link10
a link11
a link12
a link13
a link14
a link15
a link16
a link17
a link18
a link19
a link20
48
a link1
a link2
a link3
a link4
a link5
a link6
a link7
a link8
a link9
a link10
a link11
a link12
a link13
a link14
a link15
a link16
a link17
a link18
a link19
a link20
49
a link1
a link2
a link3
a link4
a link5
a link6
a link7
a link8
a link9
a link10
a link11
a link12
a link13
a link14
a link15
a link16
a link17
a link18
a link19
a link20
50
a link1
a link2
a link3
a link4
a link5
a link6
a link7
a link8
a link9
a link10
a link11
a link12
a link13
a link14
a link15
a link16
a link17
a link18
a link19
a link20
51
a link1
a link2
a link3
a link4
a link5
a link6
a link7
a link8
a link9
a link10
a link11
a link12
a link13
a link14
a link15
a link16
a link17
a link18
a link19
a link20
52
a link1
a link2
a link3
a link4
a link5
a link6
a link7
a link8
a link9
a link10
a link11
a link12
a

a link20
94
a link1
a link2
a link3
a link4
a link5
a link6
a link7
a link8
a link9
a link10
a link11
a link12
a link13
a link14
a link15
a link16
a link17
a link18
a link19
a link20
95
a link1
a link2
a link3
a link4
a link5
a link6
a link7
a link8
a link9
a link10
a link11
a link12
a link13
a link14
a link15
a link16
a link17
a link18
a link19
a link20
96
a link1
a link2
a link3
a link4
a link5
a link6
a link7
a link8
a link9
a link10
a link11
a link12
a link13
a link14
a link15
a link16
a link17
a link18
a link19
a link20
97
a link1
a link2
a link3
a link4
a link5
a link6
a link7
a link8
a link9
a link10
a link11
a link12
a link13
a link14
a link15
a link16
a link17
a link18
a link19
a link20
98
a link1
a link2
a link3
a link4
a link5
a link6
a link7
a link8
a link9
a link10
a link11
a link12
a link13
a link14
a link15
a link16
a link17
a link18
a link19
a link20
99
a link1
a link2
a link3
a link4
a link5
a link6
a link7
a link8
a link9
a link10
a link11
a link12
a link13
a link14
a